In [1]:
import pandas as pd

df=pd.read_csv("/Users/antoniojaenarias/Desktop/DATA SCIENCE/TODO/ML_KAGGLE_1/data/data.csv")
df

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8
...,...,...,...,...,...,...,...,...,...,...
31639,31639,Thompson,NaN,NaN,1736.0,0.0886,NaN,7.8,591.0,2.3
31640,31640,NaN,NaN,13.79,NaN,NaN,4.7,4.7,NaN,3.9
31641,31641,Thompson,NaN,18.27,658.0,-0.1224,3.0,3.0,150.0,2.3
31642,31642,Thompson,tube,6.89,3825.0,NaN,23.6,23.6,1972.0,3.7


In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

train = df[df['x_e_out [-]'].notnull()].reset_index(drop=True)
test = df[df['x_e_out [-]'].isnull()].reset_index(drop=True)

num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
all_features = num_cols + cat_cols

train = train[all_features]

def preprocess_data(data, cat_cols, num_cols):
    processed_data = data.copy()
    
    num_imputer = SimpleImputer(strategy='mean')
    processed_data[num_cols] = num_imputer.fit_transform(processed_data[num_cols])
    
    cat_imputer = SimpleImputer(strategy='most_frequent')
    processed_data[cat_cols] = cat_imputer.fit_transform(processed_data[cat_cols])
    
    label_encoder = LabelEncoder()
    for col in cat_cols:
        processed_data[col] = label_encoder.fit_transform(processed_data[col].astype(str))
    
    processed_data.columns = [re.sub(r'[\[\]<>\s]+', '_', col) for col in processed_data.columns]
    
    return processed_data

processed_train = preprocess_data(train, cat_cols, num_cols)



In [4]:
params = {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.9}

cv_scores = []
importance_xgb = []
preds = []
X = processed_train.drop('x_e_out_-_', axis=1)
y = processed_train['x_e_out_-_']

for i in range(3):
    print(f'\nFold {i+1} CV begins')
    skf = KFold(n_splits=3, random_state=np.random.randint(0, 999), shuffle=True)
    
    for train_ix, test_ix in skf.split(X, y):
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
             
        XGB_md = XGBRegressor(**params).fit(X_train, y_train)
        importance_xgb.append(XGB_md.feature_importances_)
        
        XGB_pred_1 = XGB_md.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, XGB_pred_1))
        cv_scores.append(rmse)
        print(f'Fold {i+1} CV done. RMSE: \033[1;36m{rmse:.5f}\033[0m')

scores = np.mean(cv_scores)    
print(f'\nThe average RMSE over 3-folds (run 3 times) is: \033[1;36m{scores:.5f}\033[0m')





Fold 1 CV begins
Fold 1 CV done. RMSE: 0.07575
Fold 1 CV done. RMSE: 0.07584
Fold 1 CV done. RMSE: 0.07291

Fold 2 CV begins
Fold 2 CV done. RMSE: 0.07552
Fold 2 CV done. RMSE: 0.07552
Fold 2 CV done. RMSE: 0.07353

Fold 3 CV begins
Fold 3 CV done. RMSE: 0.07491
Fold 3 CV done. RMSE: 0.07583
Fold 3 CV done. RMSE: 0.07423

The average RMSE over 3-folds (run 3 times) is: 0.07489


In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14153 entries, 1 to 21228
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  14153 non-null  float64
 1   pressure_MPa_       14153 non-null  float64
 2   mass_flux_kg/m2-s_  14153 non-null  float64
 3   D_e_mm_             14153 non-null  float64
 4   D_h_mm_             14153 non-null  float64
 5   length_mm_          14153 non-null  float64
 6   chf_exp_MW/m2_      14153 non-null  float64
 7   author              14153 non-null  int64  
 8   geometry            14153 non-null  int64  
dtypes: float64(7), int64(2)
memory usage: 1.1 MB


In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10415 non-null  int64  
 1   author               8794 non-null   object 
 2   geometry             8628 non-null   object 
 3   pressure [MPa]       8949 non-null   float64
 4   mass_flux [kg/m2-s]  8851 non-null   float64
 5   x_e_out [-]          0 non-null      float64
 6   D_e [mm]             8568 non-null   float64
 7   D_h [mm]             8953 non-null   float64
 8   length [mm]          8813 non-null   float64
 9   chf_exp [MW/m2]      10415 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 813.8+ KB


In [19]:
label_encoder = LabelEncoder()
for col in cat_cols:
        test[col] = label_encoder.fit_transform(test[col].astype(str))
    
        test.columns = [re.sub(r'[\[\]<>\s]+', '_', col) for col in test.columns]
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  10415 non-null  int64  
 1   author              10415 non-null  int64  
 2   geometry            10415 non-null  int64  
 3   pressure_MPa_       8949 non-null   float64
 4   mass_flux_kg/m2-s_  8851 non-null   float64
 5   x_e_out_-_          0 non-null      float64
 6   D_e_mm_             8568 non-null   float64
 7   D_h_mm_             8953 non-null   float64
 8   length_mm_          8813 non-null   float64
 9   chf_exp_MW/m2_      10415 non-null  float64
dtypes: float64(7), int64(3)
memory usage: 813.8 KB


In [20]:
X_nuevo = test.drop(columns="x_e_out_-_")
y_nuevo = test["x_e_out_-_"]



In [21]:
y_pred_nuevo = XGB_md.predict(X_nuevo)

test.rename(columns={'x_e_out_-_': 'x_e_out [-]'}, inplace=True)
test["x_e_out [-]"] = y_pred_nuevo

test.to_csv("pred.csv", index=False)



In [22]:
prueba = pd.read_csv("pred.csv")

# Mantener solo las columnas "id" y "x_e_out [mm]"
columnas_deseadas = ["id", "x_e_out [-]"]
prueba = prueba[columnas_deseadas]
# Guardar el DataFrame resultante en un nuevo archivo CSV
prueba.to_csv("pred_filtrado.csv", index=False)